In [1]:
# Data Processing
import string
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [2]:
# read RData
#res = pyreadr.read_r('5v_cleandf.RData')
!pwd

/media/csuser/DATA/ARTEMIS/romania


In [3]:
data = pd.read_csv('/media/csuser/DATA/ARTEMIS/romania/rom_triage_smote.csv')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch

2024-02-29 13:39:01.547760: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 13:39:01.678748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 13:39:02.211216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# Split the data into features (X) and target (y)
print(data.shape)
x = data.drop(['esi'], axis=1)
y = data['esi']

x = x.to_numpy().astype(np.float32)
y = y.to_numpy().reshape(-1,1).astype(np.int_)

print(np.asarray(x))

# x = tf.constant(np.asarray(x), dtype=tf.float64)
# y = tf.constant(np.asarray(y).reshape(-1, 1), dtype=tf.float64)

print(x.shape)

# convert to one hot vector
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False).fit(y)
print(ohe.categories_)

y = ohe.transform(y)
y.shape

(589260, 490)
[[0.0000000e+00 4.3750000e-01 1.6393442e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.0000000e+00 4.8750001e-01 1.3114753e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.0000000e+00 5.2499998e-01 1.6393442e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [5.8925700e+05 5.4111397e-01 1.5932502e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [5.8925800e+05 5.1393998e-01 1.3114753e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [5.8925900e+05 4.7213373e-01 1.9371408e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
(589260, 489)
[array([1, 2, 3, 4, 5])]


(589260, 5)

In [6]:
# Building a basic model
# def make_model(input_shape, num_classes):
#     inputs = keras.Input(shape=input_shape)
#     dense1 = layers.Dense(units=128, activation='LeakyReLU', use_bias=True,)(inputs)
#     dense2 = layers.Dense(units=64, activation='LeakyReLU', use_bias=True,)(dense1)
#     dense3 = layers.Dense(units=32, activation='LeakyReLU', use_bias=True,)(dense2)
#     dense4 = layers.Dense(units=128, activation='LeakyReLU', use_bias=True,)(dense3)
#     dense5 = layers.Dense(units=64, activation='LeakyReLU', use_bias=True,)(dense4)
#     dense6 = layers.Dense(units=32, activation='LeakyReLU', use_bias=True,)(dense5)
#     dense7 = layers.Dense(units=32, activation='LeakyReLU', use_bias=True,)(dense6)
#     output = layers.Dense(units=num_classes, activation='softmax')(dense7)
#     return keras.Model(inputs, output)


# # romania model
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    dense1 = layers.Dense(units=50, activation='ReLU', use_bias=True,)(inputs)
    dense2 = layers.Dense(units=50, activation='ReLU', use_bias=True,)(dense1)
    dense3 = layers.Dense(units=50, activation='ReLU', use_bias=True,)(dense2)
    dense4 = layers.Dense(units=50, activation='ReLU', use_bias=True,)(dense3)
    output = layers.Dense(units=num_classes, activation='softmax')(dense4)
    return keras.Model(inputs, output)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=1)
X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.20, random_state=1)

In [8]:
y[0].shape

(5,)

In [12]:
model = make_model(input_shape=(489), num_classes=y[0].shape[0])
model.compile(
    optimizer=keras.optimizers.Adamax(learning_rate=0.01, weight_decay=1e-6),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=["accuracy"],
)
model.summary()

# make the save callback
best_model_path = '/media/csuser/DATA/ARTEMIS/models/romania_smote'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# load the saved model by:
model.load_weights(best_model_path)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 489)]             0         
                                                                 
 dense_5 (Dense)             (None, 50)                24500     
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 dense_7 (Dense)             (None, 50)                2550      
                                                                 
 dense_8 (Dense)             (None, 50)                2550      
                                                                 
 dense_9 (Dense)             (None, 5)                 255       
                                                                 
Total params: 32,405
Trainable params: 32,405
Non-trainable

In [13]:
type(y_train)

numpy.ndarray

In [14]:
history = (model.fit(X_train, y_train, epochs=5000, validation_data=(X_val, y_val), callbacks=[model_checkpoint_callback]))
# add batch_size=128 for replicating romania model

Epoch 1/5000
11786/11786 [==============================] - 8s 667us/step - loss: 0.1734 - accuracy: 0.1992 - val_loss: 0.1600 - val_accuracy: 0.2008
Epoch 2/5000
11786/11786 [==============================] - 8s 659us/step - loss: 0.1601 - accuracy: 0.1981 - val_loss: 0.1600 - val_accuracy: 0.2011
Epoch 3/5000
 5622/11786 [=============>................] - ETA: 3s - loss: 0.1600 - accuracy: 0.2002

KeyboardInterrupt: 

In [ ]:
# CONFUSION MATRIX
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, classification_report, recall_score, ConfusionMatrixDisplay
from numpy import argmax

# load best saved model
model.load_weights(best_model_path)

y_pred = model.predict(X_test)
print(y_test.shape)
print(y_pred.shape)
y_test = argmax(a=y_test, axis=1)
y_pred = argmax(a=y_pred, axis=1)
print(y_test)
print(y_pred)
ConfusionMatrixDisplay.from_predictions(y_test+1, y_pred+1)
plt.savefig('/media/csuser/DATA/ARTEMIS/romania/figs/NN_confusion_romania_smote.png', dpi=500)

In [ ]:
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)

FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
print(TPR, TNR, PPV, NPV, FPR, FDR, ACC)

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()